In [2]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool
import requests
import pandas as pd

# Function to fetch data from FRED API
def fetch_us_unemployment_claims():
    url = 'https://api.stlouisfed.org/fred/series/observations'
    params = {
        'series_id': 'ICSA',
        'api_key': 'd9948790ea4e3cd88cc398776233755c',  # Replace with your FRED API key
        'file_type': 'json'
    }

    response = requests.get(url, params=params)
    data = response.json()

    if 'observations' in data and data['observations']:
        observations = data['observations']
        date = [entry['date'] for entry in observations]
        value = [entry['value'] for entry in observations]
        df = pd.DataFrame({'date': date, 'value': value})
        df['date'] = pd.to_datetime(df['date'])
        df['value'] = pd.to_numeric(df['value'])
        return df
    else:
        print("No data available.")
        return None

# Fetch data
unemployment_data = fetch_us_unemployment_claims()

# Plotting using Bokeh with hover tool
if unemployment_data is not None:
    output_file("unemployment_claims_plot.html")  # Output the plot to an HTML file

    hover = HoverTool(tooltips=[('Date', '@date{%F}'), ('Claims', '@value')], formatters={'@date': 'datetime'})

    p = figure(title='US Unemployment Claims', x_axis_label='Date', y_axis_label='Claims', x_axis_type='datetime',
               tools=[hover, 'pan', 'wheel_zoom', 'reset'])

    p.line(x='date', y='value', line_color='blue', line_width=2, source=unemployment_data)

    show(p)  # Opens the plot in a browser
